# Pydantic Fundamentals

In [ ]:
print("hello world")

### Start without pydantic

In [ ]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        self.name = name
        self.gender = gender
        self.age = age
        
person1 = Person(name = "Erik", age = 37, gender = "M")
person1

In [3]:
person1.name, person1.age

('Erik', 34)

In [5]:
person2 = Person(name = 3212+3213, gender = True, age="minus ten")
person2

## Validation of our Person class

In [9]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age
        


try:        
    person1 = Person(name = 32, age = 37, gender = "M")
except TypeError as err:
    print(err)
    

name must be of type str not <class 'int'>


In [ ]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age
        
    @property
    def age(self):
        return self.age
    
    
    @age.setter
    def age(self, age):    
        if not isinstance(age, int): 
            raise TypeError(f"age must be of type int not {type(age)}")
        
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124 not {age}")
        self._age = age
        
        
    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"        
        


try:        
    person3 = Person(name = "Luca", age = 126, gender = "M")
except ValueError as err:
    print(err)
    
person3

age must be between 0 and 124 not 126


Person(Luca, M, 2)

In [19]:
person4 = Person(name = "Andrea", age = 45, gender = "M")
person4

Person(Andrea, M, 45)

In [21]:
person4.age = -5
person4

Person(Andrea, M, -5)

In [29]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age
        
    @property
    def age(self):
        return self._age
    
    
    @age.setter
    def age(self, age):    
        if not isinstance(age, int): 
            raise TypeError(f"age must be of type int not {type(age)}")
        
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124 not {age}")
        self._age = age
        
        
    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"        
        
person5 = Person(name = "Alessandro", age = 50, gender = "M")
person5

Person(Alessandro, M, 50)

In [30]:
try:
    person5.age = -3
except ValueError as err:
    print(err)

age must be between 0 and 124 not -3


In [33]:
try:
    Person(name = "Fernando", age = 35, gender = "M")
except TypeError as err:
    print(err)
    
    


### Validating using pydantic

In [35]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    gender: str
    age:int
    
    
person6 = Person(name = "Olivia", age = 55, gender = "F")
person6    


Person(name='Olivia', gender='F', age=55)

In [36]:
person6.age

55

In [37]:
person6.age = 36
person6

Person(name='Olivia', gender='F', age=36)

In [39]:
person6.age = "thirty"
person6

Person(name='Olivia', gender='F', age='thirty')

In [41]:
from pydantic import ValidationError

try:
    person6 = Person(name = 2323, age = 55, gender = "F")
except ValidationError as err:
    print(err)

1 validation error for Person
name
  Input should be a valid string [type=string_type, input_value=2323, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [ ]:

# str "29" is coerced into 29
person6 = Person(name = "Olivia", age = "55", gender = "F")
person6

Person(name='Olivia', gender='F', age=55)

In [51]:
from pydantic import ConfigDict

class Person(BaseModel):
    name: str
    gender: str
    age:int
    
    model_config = ConfigDict(validate_assignment=True)
    
person7 = Person(name = "Olivia", age = "55", gender = "F")

try:
    person7.age = "thirty"
except ValidationError as err:
    print(err)
    
person7.age = 10
person7

    

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


Person(name='Olivia', gender='F', age=10)

In [ ]:
try:
    person8 = Person(name = "Sofia", age = -3, gender = "F")
except ValidationError as err:
    print(err)

## Add assigned age validation

In [58]:
from pydantic import Field
from typing import Literal

class Person(BaseModel):
    name: str
    gender: Literal["M", "F"]
    age:int = Field(gt = -1, lt = 125)
    
    model_config = ConfigDict(validate_assignment=True)
    
    
try:
    person8 = Person(name = "Sofia", age = -3, gender = "Fem")
except ValidationError as err:
    print(err)

2 validation errors for Person
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='Fem', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-3, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


### Serializtion and deserialization

In [60]:
person7.model_dump()

{'name': 'Olivia', 'gender': 'F', 'age': 10}

In [62]:
import json
with open("person.json", "w") as file:
    json.dump(person7.model_dump(), file)

In [63]:
# Deserialization
with open ("person.json", "r") as file:
    json_data = file.read()
    
print(repr(json_data))

Person.model_validate_json(json_data)

'{"name": "Olivia", "gender": "F", "age": 10}'


Person(name='Olivia', gender='F', age=10)